In [ ]:
import os
from contextlib import asynccontextmanager
from agentscope_runtime.engine import Runner
from agentscope_runtime.engine.agents.llm_agent import LLMAgent
from agentscope_runtime.engine.llms import QwenLLM
from agentscope_runtime.engine.schemas.agent_schemas import (
    MessageType,
    RunStatus,
    AgentRequest,
)
from agentscope_runtime.engine.services.context_manager import (
    ContextManager,
)

print("✅ Dependencies imported successfully")

In [ ]:
import asyncio
import os
from agentscope_runtime.engine import Runner
from agentscope_runtime.engine.agents.llm_agent import LLMAgent
from agentscope_runtime.engine.llms import QwenLLM
from agentscope_runtime.engine.schemas.agent_schemas import AgentRequest
from agentscope_runtime.engine.services.context_manager import ContextManager


async def main():
    # Set up the language model and agent
    model = QwenLLM(
        model_name="qwen-turbo",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
    )
    llm_agent = LLMAgent(model=model, name="llm_agent")

    async with ContextManager() as context_manager:
        runner = Runner(agent=llm_agent, context_manager=context_manager)

        # Create a request and stream the response
        request = AgentRequest(
            input=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "What is the capital of France?",
                        },
                    ],
                },
            ],
        )

        async for message in runner.stream_query(request=request):
            if hasattr(message, "text"):
                print(f"Streamed Answer: {message.text}")


# await main()

In [ ]:
# Create an LLM instance
model = QwenLLM(
    model_name="qwen-turbo",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# Create the LLM Agent
llm_agent = LLMAgent(
    model=model,
    name="llm_agent",
    description="A simple LLM agent for text generation",
)

print("✅ LLM Agent created successfully")

In [ ]:
from agentscope.agent import ReActAgent
from agentscope.model import OpenAIChatModel
from agentscope_runtime.engine.agents.agentscope_agent import AgentScopeAgent

agent = AgentScopeAgent(
    name="Friday",
    model=OpenAIChatModel(
        "gpt-4",
        api_key=os.getenv("OPENAI_API_KEY"),
    ),
    agent_config={
        "sys_prompt": "You're a helpful assistant named {name}.",
    },
    agent_builder=ReActAgent,
)

print("✅ AgentScope agent created successfully")

In [ ]:
# pip install "agentscope-runtime[autogen]"
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from agentscope_runtime.engine.agents.autogen_agent import AutogenAgent

agent = AutogenAgent(
    name="Friday",
    model = OpenAIChatCompletionClient(model="Qwen/Qwen3-235B-A22B", model_info={
                                                "vision": False,
                                                "function_calling": True,
                                                "json_output": True,
                                                "structured_output": False,
                                                "family": "unknown",},),
    agent_config={
        "system_message": "You're a helpful assistant",
    },
    agent_builder=AssistantAgent,
)

In [ ]:
# pip install "agentscope-runtime[langgraph]"
from typing import TypedDict
from langgraph import graph, types
from agentscope_runtime.engine.agents.langgraph_agent import LangGraphAgent


# define the state
class State(TypedDict, total=False):
    id: str


# define the node functions
async def set_id(state: State):
    new_id = state.get("id")
    assert new_id is not None, "must set ID"
    return types.Command(update=State(id=new_id), goto="REVERSE_ID")


async def reverse_id(state: State):
    new_id = state.get("id")
    assert new_id is not None, "ID must be set before reversing"
    return types.Command(update=State(id=new_id[::-1]))


state_graph = graph.StateGraph(state_schema=State)
state_graph.add_node("SET_ID", set_id)
state_graph.add_node("REVERSE_ID", reverse_id)
state_graph.set_entry_point("SET_ID")
compiled_graph = state_graph.compile(name="ID Reversal")
agent = LangGraphAgent(graph=compiled_graph)

In [ ]:
@asynccontextmanager
async def create_runner():
    async with Runner(
        agent=llm_agent,
        context_manager=ContextManager(),
    ) as runner:
        print("✅ Runner created successfully")
        yield runner

In [ ]:
async def interact_with_agent(runner):
    # Create a request
    request = AgentRequest(
        input=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "What is the capital of France?",
                    },
                ],
            },
        ],
    )

    # Stream the response
    print("🤖 Agent is processing your request...")
    all_result = ""
    async for message in runner.stream_query(request=request):
        # Check if this is a completed message
        if (
            message.object == "message"
            and MessageType.MESSAGE == message.type
            and RunStatus.Completed == message.status
        ):
            all_result = message.content[0].text

    print(f"📝 Agent response: {all_result}")
    return all_result

In [89]:
async def test_interaction():
    async with create_runner() as runner:
        await interact_with_agent(runner)

await test_interaction()

{"time": "2025-09-28 22:00:10.847", "step": "agent_step_start", "model": "", "user_id": "", "code": "", "message": "", "task_id": "", "request_id": "", "context": {"request": {"input": [{"sequence_number": null, "object": "message", "status": "created", "error": null, "id": "msg_4ff92a15-4b8c-4da4-b9ca-c9f4dbc74ac9", "type": "message", "role": "user", "content": [{"sequence_number": null, "object": "content", "status": null, "error": null, "type": "text", "index": null, "delta": false, "msg_id": null, "text": "What is the capital of France?"}], "code": null, "message": null, "usage": null}], "stream": true, "model": null, "top_p": null, "temperature": null, "frequency_penalty": null, "presence_penalty": null, "max_tokens": null, "stop": null, "n": 1, "seed": null, "tools": null, "session_id": null, "response_id": null}}, "interval": {"type": "agent_step_start", "cost": 0}, "ds_service_id": "test_id", "ds_service_name": "test_name"}
{"time": "2025-09-28 22:00:10.848", "step": "agent_ste

✅ Runner created successfully
🤖 Agent is processing your request...


{"time": "2025-09-28 22:00:11.490", "step": "agent_step_end", "model": "", "user_id": "", "code": "", "message": "", "task_id": "", "request_id": "", "context": {"request": {"input": [{"sequence_number": null, "object": "message", "status": "created", "error": null, "id": "msg_4ff92a15-4b8c-4da4-b9ca-c9f4dbc74ac9", "type": "message", "role": "user", "content": [{"sequence_number": null, "object": "content", "status": null, "error": null, "type": "text", "index": null, "delta": false, "msg_id": null, "text": "What is the capital of France?"}], "code": null, "message": null, "usage": null}], "stream": true, "model": null, "top_p": null, "temperature": null, "frequency_penalty": null, "presence_penalty": null, "max_tokens": null, "stop": null, "n": 1, "seed": null, "tools": null, "session_id": null, "response_id": null}}, "interval": {"type": "agent_step_end", "cost": "0.643"}, "ds_service_id": "test_id", "ds_service_name": "test_name"}


📝 Agent response: The capital of France is Paris.


In [ ]:
from agentscope_runtime.engine.deployers import LocalDeployManager

async def deploy_agent(runner):
    # Create deployment manager
    deploy_manager = LocalDeployManager(
        host="0.0.0.0",
        port=8090,
    )

    # Deploy the agent as a streaming service
    deploy_result = await runner.deploy(
        deploy_manager=deploy_manager,
        endpoint_path="/process",
        stream=True,  # Enable streaming responses
    )
    print(f"🚀Agent deployed at: {deploy_result}")
    print(f"🌐 Service URL: {deploy_manager.service_url}")
    print(f"💚 Health check: {deploy_manager.service_url}/health")

    return deploy_manager

In [ ]:
async def run_deployment():
    async with create_runner() as runner:
        deploy_manager = await deploy_agent(runner)

    # Keep the service running (in production, you'd handle this differently)
    print("🏃 Service is running...")

    return deploy_manager

# Deploy the agent
deploy_manager = await run_deployment()

In [93]:
async def deploy_multiple_agents():
    async with create_runner() as runner:
        # Deploy multiple agents on different ports
        deploy_manager1 = LocalDeployManager(host="0.0.0.0", port=8092)
        deploy_manager2 = LocalDeployManager(host="0.0.0.0", port=8093)

        # Deploy first agent
        result1 = await runner.deploy(
            deploy_manager=deploy_manager1,
            endpoint_path="/agent1",
            stream=True,
        )

        # Deploy second agent (you could use different runner/agent)
        result2 = await runner.deploy(
            deploy_manager=deploy_manager2,
            endpoint_path="/agent2",
            stream=True,
        )

        print(f"🚀 Agent 1deployed: {result1}")
        print(f"🚀 Agent2 deployed: {result2}")

        return deploy_manager1, deploy_manager2

# Deploy multiple agents
deploy_managers = await deploy_multiple_agents()

INFO:     Started server process [5998]
INFO:     Waiting for application startup.
INFO:     Started server process [5998]
INFO:     Application startup complete.
INFO:     Waiting for application startup.
INFO:     Application startup complete.


✅ Runner created successfully
🚀 Agent 1deployed: {'deploy_id': '161803c2-3e26-482c-a368-063d5a525e41', 'url': 'http://0.0.0.0:8092'}
🚀 Agent2 deployed: {'deploy_id': 'e41e544c-f4fc-41ab-ba37-401e67ff184c', 'url': 'http://0.0.0.0:8093'}


INFO:     Uvicorn running on http://0.0.0.0:8092 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8093 (Press CTRL+C to quit)


In [92]:
import requests


def test_deployed_agent():
    # Prepare the test payload
    payload = {
        "input": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What is the capital of France?"},
                ],
            },
        ],
        "session_id": "test_session_001",
        "user_id": "test_user_001",
    }

    print("🧪 Testing deployed agent...")

    # Test streaming responses
    try:
        response = requests.post(
            "http://localhost:8090/process",
            json=payload,
            stream=True,
            timeout=30,
        )

        print("📡 Streaming Response:")
        for line in response.iter_lines():
            if line:
                print(f"{line.decode('utf-8')}")
        print("✅ Streaming test completed")
    except requests.exceptions.RequestException as e:
        print(f"❌ Streaming test failed: {e}")

    # Test JSON responses (if available)
    try:
        response = requests.post(
            "http://localhost:8090/process",
            json=payload,
            timeout=30,
        )

        if response.status_code == 200:
            print(f"📄 JSON Response: {response.content}")
            print("✅ JSON test completed")
        else:
            print(f"⚠️ JSON endpoint returned status: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"ℹ️ JSON endpoint not available or failed: {e}")


# Run the test
test_deployed_agent()

🧪 Testing deployed agent...
❌ Streaming test failed: HTTPConnectionPool(host='localhost', port=8090): Max retries exceeded with url: /process (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x116ed3920>: Failed to establish a new connection: [Errno 61] Connection refused'))
ℹ️ JSON endpoint not available or failed: HTTPConnectionPool(host='localhost', port=8090): Max retries exceeded with url: /process (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x116f6f410>: Failed to establish a new connection: [Errno 61] Connection refused'))


In [91]:
# Check service status
print(f"Service running: {deploy_manager.is_running}")
print(f"Service URL: {deploy_manager.service_url}")

Service running: False
Service URL: None


In [90]:
async def stop_services(*_deploy_managers):
    """Stop deployed services"""
    async def _stop():
        for i, manager in enumerate(_deploy_managers):
            if manager.is_running:
                await manager.stop()
            print(f"🛑 Service {i} stopped")
    await _stop()

await stop_services(deploy_manager)

🛑 Service 0 stopped
